In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.cluster import KMeans
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('training_data/1105_ibv_input_data.csv')
df = df.dropna(subset=['MaxHistory'])

In [3]:
print(len(df))
print(df.head())

7968
   AppID  CustomerID      ApplicationDate   Age  MaxHistory  CREDIT_TO_DEBIT_RATIO_MEAN  CREDIT_TO_DEBIT_RATIO_SD  DAILY_DEBIT_AMOUNT_MEAN  DAILY_INCOME_MEAN  DAILY_INCOME_REGULAR_MEAN  DEBIT_AMOUNT_Z  DEBIT_COUNT_Z  HIGHEST_PAY_DEPOSIT_MEAN  HIGHEST_PAY_FREQUENCY  INCOME_SOURCES_COUNT  MONTH_INFLOW_MEAN  MONTH_INFLOW_SD  MONTH_OUTFLOW_MEAN  MONTH_OUTFLOW_SD  MONTHS_WITH_FEES_RATE  MONTHS_WITH_EMPLOYMENT_RATE  NO_ACTIVITY_RATE  OD_AND_NSF_FEES_DAILY  RECURRENT_COUNT  RECURRING_RATE  BALANCE_MEAN   BALANCE_SD  BALANCE_ABOVE_RATE  LAST_BALANCE_ABOVE  AVG_BALANCE_ABOVE_7D  HIGHER_BALANCE_7D  NUM_LOAN_PMT  TOTAL_LOAN_PMT_AMT  NUM_LOAN_ORIG  TOTAL_LOAN_ORIG_AMT  PAYROLL_AMOUNT  TOTAL_CREDIT  TOTAL_DEBIT  NUM_STOP_PMT  NUM_NSF  NUM_OD  NUM_RETURN_PMT  NUM_GAMBLING_PMT  AMT_GAMBLING_PMT  NUM_EI_PMT  AMT_EI_PMT  NUM_Bree  IsGood   Payin  FPDAA  is_FPDFA
0    172         132  2023-05-01 09:48:10  35.0          63                    1.023761                  0.866899               225.37941

### Standardlization

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import roc_curve, auc

In [5]:
features_modeling = ['MaxHistory', 'CREDIT_TO_DEBIT_RATIO_MEAN', 'CREDIT_TO_DEBIT_RATIO_SD', 'DAILY_DEBIT_AMOUNT_MEAN', 
                   'DAILY_INCOME_MEAN', 'DAILY_INCOME_REGULAR_MEAN', 'DEBIT_AMOUNT_Z', 'DEBIT_COUNT_Z', 
                   'HIGHEST_PAY_DEPOSIT_MEAN', 'HIGHEST_PAY_FREQUENCY', 'INCOME_SOURCES_COUNT', 'MONTH_INFLOW_MEAN',
                   'MONTH_INFLOW_SD', 'MONTH_OUTFLOW_MEAN', 'MONTH_OUTFLOW_SD', 'MONTHS_WITH_FEES_RATE',
                   'MONTHS_WITH_EMPLOYMENT_RATE', 'NO_ACTIVITY_RATE', 'OD_AND_NSF_FEES_DAILY', 'RECURRENT_COUNT', 
                   'RECURRING_RATE', 'BALANCE_MEAN', 'BALANCE_SD', 'BALANCE_ABOVE_RATE', 'LAST_BALANCE_ABOVE',
                   'AVG_BALANCE_ABOVE_7D', 'HIGHER_BALANCE_7D', 'NUM_LOAN_PMT', 'TOTAL_LOAN_PMT_AMT', 'NUM_LOAN_ORIG', 
                   'TOTAL_LOAN_ORIG_AMT', 'PAYROLL_AMOUNT', 'TOTAL_CREDIT', 'TOTAL_DEBIT', 'NUM_STOP_PMT', 'NUM_NSF',
                   'NUM_OD', 'NUM_RETURN_PMT', 'NUM_GAMBLING_PMT', 'AMT_GAMBLING_PMT', 'NUM_EI_PMT', 'AMT_EI_PMT',
                   'NUM_Bree']
x = df[df.Age>35]
y = df[df.Age>35][['FPDAA']]
#x[features_modeling].to_csv('training_data/trainingforstandardization.csv' ,index=False) # keep a copy for standardization input

In [6]:
x

,AppID,CustomerID,ApplicationDate,Age,MaxHistory,CREDIT_TO_DEBIT_RATIO_MEAN,CREDIT_TO_DEBIT_RATIO_SD,DAILY_DEBIT_AMOUNT_MEAN,DAILY_INCOME_MEAN,DAILY_INCOME_REGULAR_MEAN,DEBIT_AMOUNT_Z,DEBIT_COUNT_Z,HIGHEST_PAY_DEPOSIT_MEAN,HIGHEST_PAY_FREQUENCY,INCOME_SOURCES_COUNT,MONTH_INFLOW_MEAN,MONTH_INFLOW_SD,MONTH_OUTFLOW_MEAN,MONTH_OUTFLOW_SD,MONTHS_WITH_FEES_RATE,MONTHS_WITH_EMPLOYMENT_RATE,NO_ACTIVITY_RATE,OD_AND_NSF_FEES_DAILY,RECURRENT_COUNT,RECURRING_RATE,BALANCE_MEAN,BALANCE_SD,BALANCE_ABOVE_RATE,LAST_BALANCE_ABOVE,AVG_BALANCE_ABOVE_7D,HIGHER_BALANCE_7D,NUM_LOAN_PMT,TOTAL_LOAN_PMT_AMT,NUM_LOAN_ORIG,TOTAL_LOAN_ORIG_AMT,PAYROLL_AMOUNT,TOTAL_CREDIT,TOTAL_DEBIT,NUM_STOP_PMT,NUM_NSF,NUM_OD,NUM_RETURN_PMT,NUM_GAMBLING_PMT,AMT_GAMBLING_PMT,NUM_EI_PMT,AMT_EI_PMT,NUM_Bree,IsGood,Payin,FPDAA,is_FPDFA
1,174,134,2023-05-01 10:14:45,51.0,90,1.194366,1.061650,195.728267,205.952200,124.399500,-1.080031,-1.092375,5154.963333,15.000000,0.0,6178.5660,5346.097239,6063.5640,5259.817061,0.80,0.0,0.677778,0.790111,0.0,0.0,271.550000,350.660075,0.444444,1,0,0,4.0,905.51,0.0,0.00,14927.94,30892.83,30317.82,6,14,4,0,0,0.0,567.0,30892.83,458.0,0,0.276800,0.0,0.0
3,308,244,2023-05-04 10:52:57,39.0,90,1.100375,0.236295,230.337083,256.043333,156.048250,-1.397763,-1.185243,3267.666667,7.333333,0.0,7681.3675,3937.155763,7684.5450,4469.129000,0.75,0.0,0.844444,0.169000,0.0,0.0,448.488889,367.511767,0.677778,1,0,0,28.0,3008.98,0.0,0.00,18725.79,30725.47,30738.18,0,0,2,0,0,0.0,520.0,30725.47,500.0,0,0.968800,0.0,0.0
5,397,322,2023-05-05 13:18:39,48.0,90,0.967830,0.107725,202.259250,195.007500,129.569333,-1.454661,-1.431827,3887.080000,2.000000,0.0,5850.2250,2730.719943,6215.8525,3114.137027,0.75,0.0,0.700000,5.637000,0.0,0.0,374.283956,524.248763,0.417582,0,1,0,6.0,592.30,0.0,0.00,11661.24,23400.90,24863.41,0,4,1,4,0,0.0,363.0,23400.90,359.0,0,0.000000,1.0,1.0
6,408,331,2023-05-05 14:59:28,54.0,83,0.800258,0.541976,303.139917,302.473167,144.705000,-1.007329,-1.021757,7627.543333,5.666667,0.0,9074.1950,8062.001988,9097.3850,6030.476675,1.00,0.0,0.578313,0.000120,0.0,0.0,746.532527,1129.538380,0.340659,0,1,1,0.0,0.00,0.0,0.00,13023.45,36296.78,36389.54,0,0,1,0,0,0.0,185.0,36296.78,179.0,1,1.607700,0.0,0.0
7,463,380,2023-05-08 13:56:37,49.0,90,1.018513,0.315380,162.677417,163.823333,92.724333,-0.294173,-1.330258,2781.730000,1.750000,0.0,5039.7000,1675.655326,5043.2375,1223.147779,1.00,0.0,0.611111,0.647000,0.0,0.0,508.165934,576.882019,0.560440,1,1,0,4.0,651.66,1.0,500.00,11126.92,20158.80,20172.95,0,4,2,1,21,877.0,307.0,20158.80,258.0,0,0.541800,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7963,95469,36513,2025-07-17 11:42:47,40.0,123,1.017879,0.044147,256.448600,259.018000,0.000000,-1.511522,-1.507480,7770.540000,108.400000,0.0,7770.5400,3635.668949,7693.4580,3672.516731,0.00,0.0,0.634146,0.000000,1.0,20.0,421.812386,621.632025,0.534091,0,0,0,0.0,0.00,0.0,0.00,0.00,38852.70,38467.29,0,0,0,0,3,0.0,655.0,38852.70,525.0,0,0.418914,0.0,0.0
7964,95537,53222,2025-07-17 14:28:09,40.0,87,0.991370,0.089624,322.166500,321.232667,0.000000,-0.589348,-0.690320,9636.980000,244.000000,0.0,9636.9800,6851.458041,9664.9950,6407.797746,0.00,0.0,0.712644,0.000000,0.0,0.0,397.562386,538.674892,0.409091,0,1,1,0.0,0.00,0.0,0.00,0.00,38547.92,38659.98,0,0,0,0,268,6224.0,1237.0,38547.92,1217.0,0,0.000000,1.0,1.0
7965,95605,53257,2025-07-17 16:14:38,53.0,101,0.978858,0.047229,267.764417,283.952333,0.000000,1.281804,0.712028,3983.250000,22.000000,0.0,9353.6800,1216.358740,9570.4325,1337.729352,0.75,0.0,0.643564,3.836436,0.0,0.0,136.934205,221.972001,0.215909,1,0,1,11.0,6150.00,12.0,3340.44,0.00,37414.72,38281.73,0,0,0,0,3,0.0,593.0,37414.72,494.0,0,0.000000,1.0,1.0
7966,95720,53311,2025-07-18 07:19:32,45.0,90,0.928298,0.119392,200.958750,194.050417,0.000000,-0.563151,-0.845210,5821.512500,49.250000,0.0,5821.5125,3055.78218

In [36]:
len(x_train)

4786

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 1/4, shuffle = False, stratify = None)
scaler = StandardScaler()
scaler.fit(x_train[features_modeling])
x_train[features_modeling] = scaler.transform(x_train[features_modeling])
x_val[features_modeling] = scaler.transform(x_val[features_modeling])
x[features_modeling] = scaler.transform(x[features_modeling])

In [ ]:
# Save standardization input and output
#y.to_csv('training_data/1106_trainingoutputdata.csv' ,index=False)

In [ ]:
#Ready catboost for training
def score(val, pred):
    fpr, tpr, thresholds = roc_curve(val, pred[:,1]) 
    roc_auc = auc(fpr, tpr) 
    return roc_auc

### Model Train & Run

In [11]:
def model_evaluation(model):
    clf = model
    clf.fit(x_train[features_modeling], y_train)
    pred_dt = clf.predict_proba(x_val[features_modeling])
    pred1_dt = clf.predict_proba(x_train[features_modeling])
    print(score(y_train, pred1_dt), score(y_val, pred_dt)) 
    print(confusion_matrix(y_train.values, pred1_dt[:,1]>0.37))
    print(confusion_matrix(y_val.values, pred_dt[:,1]>0.37))
    
    def get_score1(x, y, Kfold, num_of_folds=4):
        def score(val, pred):
            fpr, tpr, thresholds = roc_curve(val, pred[:,1]) 
            roc_auc = auc(fpr, tpr) 
            return roc_auc
        sum_of_score = 0
        print('model:',model)
        for train, test in Kfold.split(x, y):
            x_t = x[features_modeling].iloc[train,:]
            y_t = y.iloc[train,:]
            x_v = x[features_modeling].iloc[test,:]
            y_v = y.iloc[test,:]
            clf = model
            clf.fit(x_t, np.array(y_t).ravel())
            pred = clf.predict_proba(x_v)
            print('kfold: ',score(y_v, pred))
            sum_of_score += score(y_v, pred)
        return sum_of_score / num_of_folds
    Kfold = StratifiedKFold(n_splits = 5, shuffle = False)
    rfmd5 = get_score1(x, y, Kfold, num_of_folds=5)
    return rfmd5

In [ ]:
def model_application_evaluation(df_, model, training = True, kmeans = 0):
    df = df_.copy()
    if training:
        df[features_modeling] = scaler.transform(df[features_modeling])
    pred = model.predict_proba(df[df.Age>35][features_modeling])[:,1]
    num_bands = 5

    pred2 = pd.DataFrame()
    pred2['Age'] = df[df.Age>35].Age
    pred2['NBScore'] = 1000 * pred
    s_score =  pred2['NBScore'].values.reshape(-1, 1)
    if training:
        kmeans = KMeans(n_clusters=num_bands, random_state=0)
        kmeans.fit(s_score)
    pred2.loc[:, 'ClusterID'] = kmeans.predict(s_score)
    band_boundary = pred2.groupby(f'ClusterID')['NBScore'].agg(['min', 'max']).sort_values(by='min')
    band_boundary['ColBand'] = list(range(1, len(band_boundary)+1, 1))
    pred2['NBBand'] = pred2['ClusterID'].map(band_boundary[['ColBand']].to_dict()['ColBand'])
    pred2['IsGood'] = df[df.Age>35]['IsGood'].values
    pred2['Payin'] = df[df.Age>35]['Payin'].values
    k = pred2.groupby('NBBand').agg({'ClusterID':'count','IsGood':['sum','mean'],'Payin':'mean'})
    k['GoodPayin'] = pred2[pred2['Payin'] > 1].groupby('NBBand')['Payin'].mean()
    k['LowPayin'] = pred2[pred2['Payin'] < 1].groupby('NBBand')['Payin'].mean()
    k.columns = ['# Loans','# IsGood', 'IsGood Rate (%)','AvgPayin','GoodPayin','LowPayin']
    k['IsGood Rate (%)'] *= 100
    k['Count Pct (%)'] = k['# Loans'] / np.sum(k['# Loans'])* 100
#     print(k.columns)
    k['Cumulative Payin ⬆️'] = [ np.sum(k.iloc[i:,3] * k.iloc[i:,0])/np.sum(k.iloc[i:,0]) for i in range(len(k))]
    if training:
        print(band_boundary)
        return kmeans, k[['AvgPayin','GoodPayin','LowPayin','# Loans', 'Count Pct (%)', '# IsGood', 'IsGood Rate (%)', 'Cumulative Payin ⬆️']]
    else: 
        return k[['AvgPayin','GoodPayin','LowPayin','# Loans', 'Count Pct (%)', '# IsGood', 'IsGood Rate (%)', 'Cumulative Payin ⬆️']]


### CatBoost to start

In [19]:
# minimal CatBoost training (quick try)
# CatBoost imports
try:
    from catboost import CatBoostClassifier, Pool, cv
    import catboost
    print("CatBoost version:", catboost.__version__)
except ModuleNotFoundError as e:
    raise SystemExit("CatBoost not installed. Run in this environment: pip install 'catboost>=1.2,<2.0'") from e



CatBoost version: 1.2.8


In [22]:
if isinstance(y_train, pd.DataFrame):
    y_train = y_train.iloc[:, 0]  

In [23]:
print(type(y_train), getattr(y_train, 'shape', None))

<class 'pandas.core.series.Series'> (4786,)


In [24]:
# y_train is 0/1 ints
class_counts = np.bincount(y_train.astype(int))
total = len(y_train)
# same formula: total/(2*c) per class (guard for zero counts)
weights = [total / (2 * c) if c > 0 else 1.0 for c in class_counts]
print("class weights:", weights)


class weights: [0.5910101259570264, 3.2469470827679783]


In [25]:
clf = CatBoostClassifier(
    iterations=700,
    early_stopping_rounds=150,
    eval_metric='F1',
    class_weights=weights,
    random_seed=42,
    verbose=100,
    loss_function='Logloss'  # binary classification (default for CatBoostClassifier)
)


In [32]:

print('model_evaluation', model_evaluation(clf))

a, training_result = model_application_evaluation(df, clf)
print('Training Result:\n', training_result)

print('Testing Result:\n')
testing_result = model_application_evaluation(x_val, clf, training=False, kmeans=a)
testing_result


Learning rate set to 0.027882
0:	learn: 0.5637308	total: 5.05ms	remaining: 3.53s
100:	learn: 0.7278341	total: 180ms	remaining: 1.06s
200:	learn: 0.7872881	total: 357ms	remaining: 886ms
300:	learn: 0.8287774	total: 574ms	remaining: 761ms
400:	learn: 0.8843005	total: 776ms	remaining: 579ms
500:	learn: 0.9263423	total: 1.16s	remaining: 460ms
600:	learn: 0.9481500	total: 1.36s	remaining: 224ms
699:	learn: 0.9640163	total: 1.6s	remaining: 0us
0.9945243360422342 0.6509657434402333
[[3389  660]
 [   1  736]]
[[1046  354]
 [ 108   88]]
model: <catboost.core.CatBoostClassifier object at 0x140a1d7d0>
Learning rate set to 0.028661
0:	learn: 0.6444016	total: 3.15ms	remaining: 2.2s
100:	learn: 0.7127438	total: 209ms	remaining: 1.24s
200:	learn: 0.7716029	total: 397ms	remaining: 986ms
300:	learn: 0.8148423	total: 575ms	remaining: 763ms
400:	learn: 0.8646292	total: 784ms	remaining: 585ms
500:	learn: 0.9080693	total: 1.03s	remaining: 408ms
600:	learn: 0.9378987	total: 1.24s	remaining: 205ms
699:	learn

,AvgPayin,GoodPayin,LowPayin,# Loans,Count Pct (%),# IsGood,IsGood Rate (%),Cumulative Payin ⬆️
NBBand,,,,,,,,
1,1.217430,1.489641,0.453853,449,28.132832,322,71.714922,1.109750
2,1.162351,1.487627,0.395628,518,32.456140,357,68.918919,1.067599
3,1.018722,1.460564,0.346354,406,25.438596,238,58.620690,0.989568
4,0.961039,1.469801,0.292550,200,12.531328,110,55.000000,0.936488
5,0.723002,1.619535,0.244851,23,1.441103,8,34.782609,0.723002


### Define functions to evaluate model performance

In [33]:
import numpy as np
import pandas as pd

def evaluate_bad_capture(
    model,
    X,
    y_fpdaa,
    *,
    features,
    scaler=None,
    volume_pcts=(0.05, 0.10),
    age_col=None,
    age_min=None,
    proba_kind="good"  # "good" if predict_proba[:,1] = P(good); "bad" if it's P(bad)
):
    """
    Evaluate if the model can 'mark' top X% highest-risk volume with FPDAA rate >= 2x average.

    Parameters
    ----------
    model : fitted classifier with predict_proba
    X : pd.DataFrame with features (unscaled or pre-scaled)
    y_fpdaa : array-like {0,1} (1 = bad/default on first payment)
    features : list[str] model feature columns
    scaler : optional StandardScaler fit on TRAIN (applied here if provided)
    volume_pcts : iterable of floats (fractions, e.g., 0.05, 0.10)
    age_col : optional str column name for raw age
    age_min : optional numeric; if set, filter to Age > age_min
    proba_kind : "good" if proba[:,1] is P(good); "bad" if proba[:,1] is P(bad)

    Returns
    -------
    pd.DataFrame with metrics per volume slice.
    """
    df = X.copy()
    y = pd.Series(y_fpdaa).astype(int).reset_index(drop=True)

    # Optional Age filter (always on RAW age)
    if age_col is not None and age_min is not None:
        m = df[age_col] > age_min
        df = df.loc[m].reset_index(drop=True)
        y = y.loc[m.values].reset_index(drop=True)

    # Ensure feature order + (optional) scaling
    Xf = df[features].copy()
    if scaler is not None:
        Xf = pd.DataFrame(scaler.transform(Xf), columns=features, index=df.index)

    # Probabilities
    proba = model.predict_proba(Xf)[:, 1]
    p_good = proba if proba_kind == "good" else (1.0 - proba)
    risk = 1.0 - p_good  # higher = riskier

    n = len(df)
    overall_rate = y.mean() if n > 0 else np.nan

    # Sort by risk descending
    order = np.argsort(-risk)
    y_sorted = y.iloc[order].to_numpy()
    p_sorted = p_good[order]

    rows = []
    cum_bad = y_sorted.cumsum()
    total_bad = int(cum_bad[-1]) if n > 0 else 0

    for pct in volume_pcts:
        k = int(np.ceil(pct * n))
        if k < 1:
            rows.append({
                "volume_pct": pct, "slice_size": 0, "fpdaa_rate": np.nan,
                "overall_fpdaa_rate": overall_rate, "lift": np.nan,
                "bad_in_slice": 0, "total_bad": total_bad,
                "recall_of_bad": np.nan, "p_good_threshold": np.nan
            })
            continue

        # Top-k riskiest slice
        y_slice = y_sorted[:k]
        fpdaa_rate = y_slice.mean() if k > 0 else np.nan
        lift = (fpdaa_rate / overall_rate) if overall_rate and overall_rate > 0 else np.nan
        bad_in_slice = int(y_slice.sum())
        recall_of_bad = (bad_in_slice / total_bad) if total_bad > 0 else np.nan

        # Threshold on P(good) to capture this slice (lowest p_good in the slice)
        p_good_threshold = float(p_sorted[:k].max())  # boundary value for inclusion

        rows.append({
            "volume_pct": pct,
            "slice_size": k,
            "fpdaa_rate": fpdaa_rate,
            "overall_fpdaa_rate": overall_rate,
            "lift": lift,
            "bad_in_slice": bad_in_slice,
            "total_bad": total_bad,
            "recall_of_bad": recall_of_bad,
            "p_good_threshold": p_good_threshold
        })

    return pd.DataFrame(rows).assign(
        meets_2x_criterion=lambda d: d["lift"] >= 2.0
    )


In [34]:
res = evaluate_bad_capture(
    model=clf,          # your fitted CatBoost (trained with FPDAA as target=1)
    X=x_val,                      # validation features (unscaled or pass scaler below)
    y_fpdaa=x_val["FPDAA"].astype(int),  # ground-truth: 1 = bad
    features=features_modeling,
    scaler=scaler,                # pass ONLY if the model expects scaled inputs
    volume_pcts=(0.05, 0.10),     # test 5% and 10% highest-risk slices
    age_col="Age",                # optional
    age_min=35,                   # optional
    proba_kind="bad"              # <-- key change: model outputs P(bad)
)
print(res)


   volume_pct  slice_size  fpdaa_rate  overall_fpdaa_rate      lift  bad_in_slice  total_bad  recall_of_bad  p_good_threshold  meets_2x_criterion
0        0.05          80     0.16250            0.122807  1.323214            13        196       0.066327          0.463764               False
1        0.10         160     0.16875            0.122807  1.374107            27        196       0.137755          0.504962               False
